<a href="https://colab.research.google.com/github/MohamedHesham02/Sentiment-Analysis-Arabic-Tweets/blob/main/sentiment_analysis_data_preprocessing_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
import sklearn 
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/final.csv')  
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,dialect,tweet
0,0,0,0,1175358310087892992,IQ,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,1,1,1,1175416117793349632,IQ,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,2,2,2,1175450108898565888,IQ,@KanaanRema مبين من كلامه خليجي
3,3,3,3,1175471073770573824,IQ,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,4,4,4,1175496913145217024,IQ,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


In [4]:
del data['Unnamed: 0']
del data['Unnamed: 0.1']
del data['Unnamed: 0.1.1']

In [5]:
data = data.take(np.random.permutation(len(data))[:])

In [6]:
data.shape

(20833, 3)

In [7]:
data.isna().sum()

id         0
dialect    0
tweet      0
dtype: int64

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20833 entries, 10531 to 2993
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       20833 non-null  int64 
 1   dialect  20833 non-null  object
 2   tweet    20833 non-null  object
dtypes: int64(1), object(2)
memory usage: 651.0+ KB


# Text Pre-processing of Arabic letters

In [9]:
# Arabic letters encodings

COMMA = u'\u060C'
SEMICOLON = u'\u061B'
QUESTION = u'\u061F'
HAMZA = u'\u0621'
ALEF_MADDA = u'\u0622'
ALEF_HAMZA_ABOVE = u'\u0623'
WAW_HAMZA = u'\u0624'
ALEF_HAMZA_BELOW = u'\u0625'
YEH_HAMZA = u'\u0626'
ALEF = u'\u0627'
TEH_MARBUTA = u'\u0629'
TATWEEL = u'\u0640'
LAM = u'\u0644'
HEH = u'\u0647'
YEH = u'\u064a'
ALEF_MAKSURA = u'\u0649'
HAMZA_ABOVE = u'\u0654'
HAMZA_BELOW = u'\u0655'
PERCENT = u'\u066a'
DECIMAL = u'\u066b'
THOUSANDS = u'\u066c'
STAR = u'\u066d'
FULL_STOP = u'\u06d4'
BYTE_ORDER_MARK = u'\ufeff'
MULITIPLICATION_SIGN = u'\u00D7'
DIVISION_SIGN = u'\u00F7'


# Tashkeel
FATHATAN = u'\u064b'
DAMMATAN = u'\u064c'
KASRATAN = u'\u064d'
FATHA = u'\u064e'
DAMMA = u'\u064f'
KASRA = u'\u0650'
SHADDA = u'\u0651'
SUKUN = u'\u0652'

#Ligatures
LAM_ALEF = u'\ufefb'
LAM_ALEF_HAMZA_ABOVE = u'\ufef7'
LAM_ALEF_HAMZA_BELOW = u'\ufef9'
LAM_ALEF_MADDA_ABOVE = u'\ufef5'

HARAKAT_PAT = re.compile(u"["+u"".join([FATHATAN, DAMMATAN, KASRATAN, FATHA, DAMMA, KASRA, SUKUN, SHADDA])+u"]")
HAMZAT_PAT = re.compile(u"["+u"".join([WAW_HAMZA, YEH_HAMZA])+u"]")
ALEFAT_PAT = re.compile(u"["+u"".join([ALEF_MADDA, ALEF_HAMZA_ABOVE, ALEF_HAMZA_BELOW, HAMZA_ABOVE, HAMZA_BELOW])+u"]")
LAMALEFAT_PAT = re.compile(u"["+u"".join([LAM_ALEF, LAM_ALEF_HAMZA_ABOVE, LAM_ALEF_HAMZA_BELOW, LAM_ALEF_MADDA_ABOVE])+u"]")

WESTERN_ARABIC_NUMERALS = ['0','1','2','3','4','5','6','7','8','9']
EASTERN_ARABIC_NUMERALS = [u'\u06F0', u'\u06F1', u'\u06F2', u'\u06F3', u'\u0664', u'\u06F5', u'\u0666', u'\u06F7', u'\u06F8', u'\u06F9']

eastern_to_western_numerals = {}
for i in range(len(EASTERN_ARABIC_NUMERALS)):
    eastern_to_western_numerals[EASTERN_ARABIC_NUMERALS[i]] = WESTERN_ARABIC_NUMERALS[i]

arabic_punctuations = COMMA + SEMICOLON + QUESTION + PERCENT + DECIMAL + THOUSANDS + STAR + FULL_STOP + MULITIPLICATION_SIGN + DIVISION_SIGN
all_punctuations = string.punctuation + arabic_punctuations + '()[]{}' # English & Arabic punctuations

all_punctuations = ''.join(list(set(all_punctuations)))

# Text Cleaning 
### Removal of unrequired terms of sentences that could have a negative effect on the training process (e.g: tags, hashtags, urls, email addresses, .....)

In [10]:
def strip_tashkeel(text):
    text = HARAKAT_PAT.sub('', text)
    return text 

def strip_tatweel(text):
    return re.sub(u'[%s]' % TATWEEL, '', text)

def remove_non_arabic(text):
    return ' '.join(re.sub(u"[^\u0621-\u063A\u0640-\u0652 ]", " ", text,  flags=re.UNICODE).split())

def keep_arabic_english_n_symbols(text):
    return ' '.join(re.sub(u"[^\u0621-\u063A\u0640-\u064aa-zA-Z#@_:/ ]", "", text,  flags=re.UNICODE).split())

def normalize_hamza(text):
    text = ALEFAT_PAT.sub(ALEF, text)
    text = HAMZAT_PAT.sub(HAMZA, text)
    return text

def normalize_lamalef(text): 
    text = LAMALEFAT_PAT.sub((LAM, ALEF), text)
    return text

def normalize_spellerrors(text):
    text = re.sub(u'[%s]' % TEH_MARBUTA, HEH, text)
    return re.sub(u'[%s]' % ALEF_MAKSURA, YEH, text)

def remove_underscore(text):
    return ' '.join(text.split('_'))

def remove_retweet_tag(text):
    tag = re.compile('rt @[a-zA-Z0-9_]+:|@[a-zA-Z0-9_]+')
    text = tag.sub('', text)
    hashtag = re.compile('\#')
    text = hashtag.sub('', text)
    return text

def replace_emails(text):
    emails = re.findall(r'[\w\.-]+@[\w\.-]+', text)
    for email in emails:
        text = text.replace(email,'#')
    return text

def replace_urls(text):
    return re.sub(r"http\S+|www.\S+", "#", text)

def convert_eastern_to_western_numerals(text):
    for num in EASTERN_ARABIC_NUMERALS:
        text = text.replace(num, eastern_to_western_numerals[num])
    return text

def remove_all_punctuations(text):
    for punctuation in all_punctuations:
        text = text.replace(punctuation, ' ')
    return text

def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags 
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub('', text)
    return text

def replace_phone_numbers(text):
    return re.sub(r'\d{10}', '#', text)

def remove_extra_spaces(text):
    return ' '.join(text.split())

def normalize_tweet(text):
    new_text = text.lower()
    new_text = normalize_hamza(new_text)
    new_text = strip_tashkeel(new_text)
    new_text = strip_tatweel(new_text)
    new_text = normalize_spellerrors(new_text)
    new_text = remove_retweet_tag(new_text)
    new_text = replace_emails(new_text)
    new_text = remove_underscore(new_text)
    new_text = replace_phone_numbers(new_text)
    new_text = remove_all_punctuations(new_text)
    new_text = replace_urls(new_text)
    new_text = convert_eastern_to_western_numerals(new_text)
    new_text = keep_arabic_english_n_symbols(new_text)
    new_text = remove_non_arabic(new_text)
    new_text = remove_extra_spaces(new_text)
    
    return new_text

In [11]:
test= " لقد قال أنه يأكل لأنه يؤكل"

def normalize_hamza(text):
    text = ALEFAT_PAT.sub(ALEF,text)
    text = HAMZAT_PAT.sub(HAMZA,text)
    return text

result = normalize_hamza(test)
print(result)

 لقد قال انه ياكل لانه يءكل


In [12]:
data['tweet'] = data['tweet'].apply(normalize_tweet)
data.head()

,id,dialect,tweet
10531,1132446517996998784,SA,الله يصلحه يموت بالاثاره زي خوينا والكتب اثرت ...
12916,1015690276344619008,DZ,اسحب كلاااامي الله عليك يا ابو شعععر طويلللل
18942,1098503953204105216,AE,انا من الناس اللي نشرو اللعبه
20498,1027083124545646592,BH,لازم يصير شي حلو اليوم لان التاريخ ما يتطوف
10175,950283002726240384,SA,الله يعين اللي في الهوي قلبه تعلق


In [13]:
data['tweet'][5]

'يااخي الارهابي اذا كان عراقي سعودي فلسطيني وين المشكله علي باب الفرض خليجي وماعنده رحمه وين المساس بقدسيتك الله يرضي عنك خلصت مشاكل العرب واختلفنا بجنسيت ابو الطفل المغردين سالوا من وين ورجحت وين الكارثه اللي وصلتك'

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
# convert tweets col to one string
tweets = data.tweet.str.cat(sep=' ')
tweets

'الله يصلحه يموت بالاثاره زي خوينا والكتب اثرت عليه اسحب كلاااامي الله عليك يا ابو شعععر طويلللل انا من الناس اللي نشرو اللعبه لازم يصير شي حلو اليوم لان التاريخ ما يتطوف الله يعين اللي في الهوي قلبه تعلق عمول تاغ لصحبتكلبناني يقبش كلمه تشظي تكتب بالضاد ابو عصا هكذا تشظي ما يكتبوهاش الناس صلعه مثل راسك هكذا تشضي مثل ما انت كتبت في تغريدتك والا احسن اكتبها بالمصري تشزي يا سعاده الوزير راجع عقلك كماقد شكله ضيع حتي اللغه جنب الحاجات الثانيه اللي بالي بالك لو من المصنع تي مع الغطا اللي يحمي الشاشه عسب مانزيد عليها لا قلت حبيبتي شوفي الصور تفهمين بس كلامك موتني ضحك ههههههههههههههه من حقي اصف سطور ضحك فساد علني للعيان وبدون عقاب للاسف صج صج صدفه ترد الروح الله يكثر من هالصدف ونلاقيكم من قال لك انه ينتهي اساسا الشباب ما بيفوتو الفرصه ان شاء الله اليوم و بنفوز اه تقريبا دول لي فلتر لي كيعجبوني ملي نشوفوهم فالصور شكرا لك قصقص مابيجامل هاي من اخلاقها ربي يحفظها ويخلينا ياه يارب يبيله والله يمكن بالغلط احد يستوعب ان الموضوع مو بالصوره الماخوذه عنه وغنت الاوجاع الحزن والالم معه عرب وين وطنبوره وين

## Getting a frequency insight of words



In [16]:
import nltk
from nltk.tokenize import word_tokenize

#function to split text into word
tokens = word_tokenize(tweets)

vocabulary = set(tokens)
print('no. of words is:', len(vocabulary))

frequency_dist = nltk.FreqDist(tokens) # words frequency
sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:20] # highest 20 word occurred only 

no. of words is: 64108


['من',
 'في',
 'ما',
 'علي',
 'اللي',
 'الله',
 'و',
 'بس',
 'يا',
 'كل',
 'انا',
 'والله',
 'ولا',
 'لا',
 'مش',
 'شي',
 'ان',
 'مع',
 'هذا',
 'انت']

## Stopwords removal from all tweets that will not be an important paramter to classify the tweet

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
stop = stopwords.words('arabic')
arab_stopwords = set(nltk.corpus.stopwords.words("arabic"))
arab_stopwords

{'ء',
 'ءَ',
 'آ',
 'آب',
 'آذار',
 'آض',
 'آمينَ',
 'آناء',
 'آنفا',
 'آه',
 'آها',
 'آهاً',
 'آهٍ',
 'آهِ',
 'آي',
 'أ',
 'أبدا',
 'أبريل',
 'أبو',
 'أبٌ',
 'أجل',
 'أجمع',
 'أحد',
 'أخبر',
 'أخذ',
 'أخو',
 'أخٌ',
 'أربع',
 'أربعاء',
 'أربعة',
 'أربعمئة',
 'أربعمائة',
 'أرى',
 'أسكن',
 'أصبح',
 'أصلا',
 'أضحى',
 'أطعم',
 'أعطى',
 'أعلم',
 'أغسطس',
 'أف',
 'أفريل',
 'أفعل به',
 'أفٍّ',
 'أقبل',
 'أقل',
 'أكتوبر',
 'أكثر',
 'أل',
 'ألا',
 'ألف',
 'ألفى',
 'أم',
 'أما',
 'أمام',
 'أمامك',
 'أمامكَ',
 'أمد',
 'أمس',
 'أمسى',
 'أمّا',
 'أن',
 'أنا',
 'أنبأ',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'أنتِ',
 'أنشأ',
 'أنى',
 'أنًّ',
 'أنّى',
 'أهلا',
 'أو',
 'أوت',
 'أوشك',
 'أول',
 'أولئك',
 'أولاء',
 'أولالك',
 'أوه',
 'أوّهْ',
 'أى',
 'أي',
 'أيا',
 'أيار',
 'أيضا',
 'أيلول',
 'أين',
 'أينما',
 'أيها',
 'أيّ',
 'أيّان',
 'أُفٍّ',
 'ؤ',
 'إحدى',
 'إذ',
 'إذا',
 'إذاً',
 'إذما',
 'إذن',
 'إزاء',
 'إلا',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'إليكنّ',
 'إليكَ',
 'إلَيْكَ',
 'إلّا',


In [19]:
arab_stopwords = '|'.join(arab_stopwords)
arab_stopwords

'ذان|هَاتانِ|الألى|حين|وهو|عند|أوّهْ|لكيلا|آهٍ|إى|ا|زاي|آناء|هما|أفعل به|بعض|هيا|قام|أيها|منه|ما برح|وإذ|كلما|به|كأيّ|أربع|بما|إذما|قد|اربعين|أن|تسعة|أنتن|تلكما|يورو|ساء|د|أيضا|عجبا|لها|فلس|هنا|أصلا|إليكن|هَذانِ|بين|منها|وجد|ذا|مع|أوشك|خاصة|أكتوبر|فإذا|لعلَّ|إذاً|ذوا|ذلكن|إذا|إنا|ب|جمعة|آها|ليس|أقل|ميم|عليه|ها|ليست|جويلية|هاتي|يا|التي|ث|كذا|وإذا|ثمانية|ثمنمئة|اللواتي|أين|طرا|ستة|ما|طاق|ثلاثون|لدن|أنتما|هاته|صراحة|لما|وهب|طَق|ياء|لكنما|ذلكم|أبو|لنا|بيد|صار|تسع|لك|نوفمبر|سبعون|بعد|كأيّن|إيانا|كِخ|كى|عاد|عشرون|مادام|ت|خمسة|هَاتِي|ض|تشرين|إليكَ|هاء|مساء|ذانِ|ارتدّ|أخبر|رزق|درهم|بخ|لي|والذي|إلَيْكَ|اخلولق|صاد|حار|كلاهما|ثمان|شرع|دال|خلف|ليت|تاء|فلان|لوما|أُفٍّ|راء|آب|آذار|ز|كأن|ءَ|ذِه|ذاك|بئس|هَاتِه|إنَّ|ثم|كاد|عشر|ألا|أولاء|اثنان|سبعمئة|راح|صهٍ|تَيْنِ|ماذا|دون|سحقا|بكن|مائة|همزة|تحوّل|هكذا|هذين|تعلَّم|تِه|أنت|مكانَك|شتانَ|سوى|ح|أجل|غدا|غالبا|اللتيا|أعلم|أقبل|لستم|أرى|تعسا|إياهن|ألف|أنًّ|حتى|تلقاء|ديسمبر|نفس|هبّ|ثامن|عاشر|شمال|من|ولا|أوه|ولكن|اربعون|ترك|هؤلاء|إياكن|عدَّ|جير|واو|هيت|إذ|فبراي

In [20]:
data['tweet']=data['tweet'].replace(arab_stopwords, '')
data['tweet'].tail(10)

4912     اللبنانيين بيحكو بناني مو عربي تطور الموضوع وص...
14252    انت مسجله حضوووورفي ناس كانت متابعانيوتويتر لغ...
19437    مش مشهوره معناه اخطي ونعدي لك انت فدوله فيها س...
12784                     هدي اختوتي تاع الجواجله تاع الصح
4128                                   امين يا رب يسمع منك
4055         عن لما يبطل يفرق معك اشي بالدنيا شعور التمسحه
13921    انا تقريبا بجد والله نمت من كتر النكد كاني روح...
9909                   مصممه فلاتر ادعموها الله يجزاكم خير
15006    بعد نهايه الميركاتو بسالك عن رايك بصيف اليوناي...
2993     مسكين والله دخلت علي الردود عنده ماشفت رد سنعك...
Name: tweet, dtype: object

## Data Preparation 

### Tokenization 
Coverting sentences to vector of sentence words 

### Stemming is a process of converting every word to its root of meaning 
 This process will help the model gain the dialect of each country and attitude of writing on twitter for easier classification

In [21]:
# small example of stemming

from nltk.stem.isri import ISRIStemmer
st = ISRIStemmer()

token = word_tokenize(data['tweet'][15])
print(token)
string = []

for i in token:
    stemmed = st.stem(i)
    string.append(stemmed)
    fstring = ' '.join(string)
print(fstring)

['يسعد', 'مساك', 'بنت', 'العم']
سعد ساك بنت لعم


In [22]:
# Applying same process on all dataset

for a in range(len(data['tweet'])):
    token = word_tokenize(data['tweet'][a])
    print(token)
    string = []
    for i in token:
        stemmed = st.stem(i)
        string.append(stemmed)
    fstring = ' '.join(string)
    data['tweet'][a] = fstring
    print(data['tweet'][a])

Streaming output truncated to the last 5000 lines.
['شكلك', 'لقيتها', 'شينه', 'وخجلت', 'منها', 'اولا', 'انا', 'ما', 'بديت', 'اي', 'كلام', 'معاكم', 'لقيت', 'حوار', 'جاري', 'بطريقه', 'مستفزه', 'جدا', 'ودخلت', 'فيه', 'وثانيا', 'انا', 'ما', 'شتمتك', 'ولا', 'مره', 'بالرغم', 'انك', 'شتمتني', 'وقلته', 'قليله', 'ادب', 'لكن', 'الحمدلله', 'تربيتي', 'منعتني', 'اجاريك', 'في', 'الاساءه', 'وكنت', 'برد', 'علي', 'تغريداتك', 'فقط']
شكل لقت شين خجل منها اول انا ما بدت اي كلم معا لقت حور جري طرق فزه جدا دخل فيه وثن انا ما شتم ولا مره رغم انك شتم وقل قلل ادب لكن حمدلل ربت نعت جري في سءه وكن برد علي تغريداتك فقط
['ويكيبديا', 'دي', 'يا', 'ماما', 'مش', 'مصدر', 'موثوق', 'لمعلوماتك', 'المحدوده', 'جدا', 'اي', 'حد', 'ممكن', 'يعمل', 'حساب', 'و', 'يغير', 'ويبدل', 'فيها', 'بليها', 'واشربي', 'ميتها', 'يا', 'ميوووووووت']
كيبد دي يا اما مش صدر وثق لمعلوماتك حدد جدا اي حد مكن عمل حسب و يغر بدل فيه بلي شرب ميت يا ميوووووووت
['اتامرون', 'الناس', 'بالبر', 'وتنسون', 'انفسكم']
تمر ناس لبر تنس نفس
['صدقني', 'دي', 'الحقيقيه',

In [23]:
data.head(5)

,id,dialect,tweet
10531,1132446517996998784,SA,الل صلح يمت ثره زي خوي كتب اثر عليه
12916,1015690276344619008,DZ,سحب كلاااامي الل علك يا ابو شعععر طويلللل
18942,1098503953204105216,AE,انا من ناس الل شرو لعب
20498,1027083124545646592,BH,لزم يصر شي حلو اليوم لان ارخ ما تطف
10175,950283002726240384,SA,الل يعن الل في هوي قلب علق


## Word Embedding is conversion into a numerical format where each tweet is represented by a matrix (word vectors)

Term Frequency – Inverse document frequency (TF-IDF) is an embedding technique that is intended to reflect how important a word is to a document  The tf–idf value increases proportionally to the number of times a word appears in the document 



Term Frequency (TF) = (Frequency of a term in the document)/(Total number of terms in documents)
Inverse Document Frequency(IDF) = log( (total number of documents)/(number of documents with term t))

TF-IDF = (TF).(IDF)

## N-Gram Concept
n-grams is a sequence of N items from a given example of text to get the highest probablities of followed sequence of words can form a specific dialect or class 

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer                                                 # min. n & max. n
unigram_word_vectorizer = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode', analyzer='word', ngram_range=(1, 1), max_features=10000)

In [25]:
unigramdata = unigram_word_vectorizer.fit_transform(data['tweet'].astype('str'))
unigramdata = unigramdata.toarray()

vocab = unigram_word_vectorizer.get_feature_names()
unigrams = pd.DataFrame(np.round(unigramdata, 1), columns=vocab)
unigrams[unigrams > 0] = 1

unigrams.head()

,ءبر,ءبررر,ءبرررر,ءبش,ءبشو,ءثر,ءخر,ءدب,ءسس,ءش,...,ييز,ييس,ييش,ييض,ييغ,ييك,ييم,يين,ييه,ييي
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
import joblib

joblib.dump(unigram_word_vectorizer, 'vectorizer_feature.pkl')

['vectorizer_feature.pkl']

In [ ]:
unigrams.to_csv('/content/drive/MyDrive/unigrams_features.csv')

## Another Word Embedding Technique
CountVectorizer is converting a collection of text documents to a matrix of word counts.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(stop_words= stopwords.words("arabic"),ngram_range=(2, 2), max_features = 10000)
X = count_vectorizer.fit_transform(data['tweet']).toarray()
df_bigrams = pd.DataFrame(X, columns=count_vectorizer.get_feature_names())
df_bigrams.shape

(20833, 10000)

In [ ]:
df_bigrams.to_csv('/content/drive/MyDrive/bigrams_features.csv')

In [ ]:
v = CountVectorizer(ngram_range=(2, 2))
print(v.fit(["an apple a day keeps the doctor away"]).vocabulary_)

{'an apple': 0, 'apple day': 1, 'day keeps': 2, 'keeps the': 4, 'the doctor': 5, 'doctor away': 3}


In [ ]:
count_vectorizer = CountVectorizer(stop_words= stopwords.words("arabic"),ngram_range=(3, 3), max_features = 10000)
X = count_vectorizer.fit_transform(data['tweet']).toarray()
df_trigram = pd.DataFrame(X, columns=count_vectorizer.get_feature_names())
df_trigram.shape

(20833, 10000)

In [ ]:
df_trigram.to_csv('/content/drive/MyDrive/trigrams_features.csv')

## Label Encoding 
converting classes into numbers 

In [ ]:
from sklearn import preprocessing

pro = preprocessing.LabelEncoder()
encpro = pro.fit_transform(data['dialect'])
data['dialect'] = encpro
y = data['dialect']

In [ ]:
y.to_csv('/content/drive/MyDrive/labels.csv')

In [ ]:
y.shape

(20833,)

### Text Sequence 
vectorizing all tweets as sequence of features and making the highest existing words in corpus having higher value 

### Padding Sequence
framming the vector by zeros to make all sentences vectors of same length

In [ ]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import text,sequence

X=data['tweet'].values

max_features=10000
x_tokenizer=Tokenizer(max_features)
x_tokenizer.fit_on_texts(X)
X = x_tokenizer.texts_to_sequences(X) 
X = sequence.pad_sequences(X)

In [ ]:
X.shape

(20833, 58)

In [ ]:
seq_data = pd.DataFrame(X)
seq_data.to_csv('/content/drive/MyDrive/seq_data.csv')